In [51]:
import pandas as pd 

In [52]:
player_data_merged=pd.read_excel(r"C:\Users\Daphne\OneDrive\Documents\NBA data set\NBA_player_data_new.xlsx")

In [53]:
player_data_merged

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_ID,PLAYER_NAME,MIN,...,PF,PTS,PLUS_MINUS,SEASON,IS_HOME,TEAM_WINS,TEAM_CITY,TS%,GAME_DATE_EST,eFG%
0,0,0,0,0,22200477,1610612759,SAS,1629641,Romeo Langford,18:06,...,5,2,-2,2022,0,0,San Antonio,1.000,2022-12-22,1.000
1,1,1,1,1,22200477,1610612759,SAS,1631110,Jeremy Sochan,31:01,...,1,23,-14,2022,0,0,San Antonio,0.625,2022-12-22,0.571
2,2,2,2,2,22200477,1610612759,SAS,1627751,Jakob Poeltl,21:42,...,4,13,-4,2022,0,0,San Antonio,0.689,2022-12-22,0.667
3,3,3,3,3,22200477,1610612759,SAS,1630170,Devin Vassell,30:20,...,1,10,-18,2022,0,0,San Antonio,0.372,2022-12-22,0.346
4,4,4,4,4,22200477,1610612759,SAS,1630200,Tre Jones,27:44,...,2,19,0,2022,0,0,San Antonio,0.690,2022-12-22,0.625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246066,246066,246066,246066,472610,11400001,1610612748,MIA,203920,Khem Birch,16:47,...,2,4,1,2014,1,0,Miami,0.515,2014-10-04,0.667
246067,246067,246067,246067,472611,11400001,1610612748,MIA,204053,Shawn Jones,15:19,...,2,2,-6,2014,1,0,Miami,0.333,2014-10-04,0.333
246068,246068,246068,246068,472612,11400001,1610612748,MIA,203894,Shabazz Napier,7:30,...,2,5,-7,2014,1,0,Miami,0.833,2014-10-04,0.833
246069,246069,246069,246069,472613,11400001,1610612748,MIA,200769,Shannon Brown,5:53,...,0,1,-3,2014,1,0,Miami,0.266,2014-10-04,0.000


In [54]:
# Convert GAME_DATE_EST to datetime for accurate sorting
player_data_merged['GAME_DATE_EST'] = pd.to_datetime(player_data_merged['GAME_DATE_EST'])

# Sort by PLAYER_ID and GAME_DATE_EST
player_data_merged = player_data_merged.sort_values(by=['PLAYER_ID', 'GAME_DATE_EST'])

# Convert 'MIN' to a numerical value (total minutes as a float)
def convert_minutes_to_float(min_str):
    try:
        if ':' in min_str:
            minutes, seconds = min_str.split(':')
            return int(minutes) + int(seconds) / 60
        else:
            return float(min_str)
    except ValueError:
        return 0  # Handle any unexpected format or missing data

player_data_merged['MIN'] = player_data_merged['MIN'].apply(convert_minutes_to_float)

# Calculate rolling averages for the last 10 games for each player
player_data_merged['avg_min'] = player_data_merged.groupby('PLAYER_ID')['MIN'].transform(lambda x: x.shift(1).rolling(window=10, min_periods=1).mean())
player_data_merged['avg_pts'] = player_data_merged.groupby('PLAYER_ID')['PTS'].transform(lambda x: x.shift(1).rolling(window=10, min_periods=1).mean())
player_data_merged['avg_reb'] = player_data_merged.groupby('PLAYER_ID')['REB'].transform(lambda x: x.shift(1).rolling(window=10, min_periods=1).mean())
player_data_merged['avg_ast'] = player_data_merged.groupby('PLAYER_ID')['AST'].transform(lambda x: x.shift(1).rolling(window=10, min_periods=1).mean())
player_data_merged['avg_stl'] = player_data_merged.groupby('PLAYER_ID')['STL'].transform(lambda x: x.shift(1).rolling(window=10, min_periods=1).mean())
player_data_merged['avg_blk'] = player_data_merged.groupby('PLAYER_ID')['BLK'].transform(lambda x: x.shift(1).rolling(window=10, min_periods=1).mean())
player_data_merged['avg_to'] = player_data_merged.groupby('PLAYER_ID')['TO'].transform(lambda x: x.shift(1).rolling(window=10, min_periods=1).mean())
player_data_merged['avg_pf'] = player_data_merged.groupby('PLAYER_ID')['PF'].transform(lambda x: x.shift(1).rolling(window=10, min_periods=1).mean())

player_data_merged['avg_fg_pct'] = player_data_merged.groupby('PLAYER_ID')['FG_PCT'].transform(lambda x: x.shift(1).rolling(window=10, min_periods=1).mean())
player_data_merged['avg_fg3_pct'] = player_data_merged.groupby('PLAYER_ID')['FG3_PCT'].transform(lambda x: x.shift(1).rolling(window=110, min_periods=1).mean())
player_data_merged['avg_ft_pct'] = player_data_merged.groupby('PLAYER_ID')['FT_PCT'].transform(lambda x: x.shift(1).rolling(window=10, min_periods=1).mean())

player_data_merged['avg_plus_minus'] = player_data_merged.groupby('PLAYER_ID')['PLUS_MINUS'].transform(lambda x: x.shift(1).rolling(window=10, min_periods=1).mean())
player_data_merged['avg_ts_pct'] = player_data_merged.groupby('PLAYER_ID')['TS%'].transform(lambda x: x.shift(1).rolling(window=10, min_periods=1).mean())
player_data_merged['avg_efg_pct'] = player_data_merged.groupby('PLAYER_ID')['eFG%'].transform(lambda x: x.shift(1).rolling(window=10, min_periods=1).mean())

# Fill missing values using forward fill
player_data_merged = player_data_merged.fillna(method='ffill')

# If there are still NaNs (for the very first match), fill them with 0
player_data_merged = player_data_merged.fillna(0)

C:\Users\Daphne\AppData\Local\Temp\ipykernel_22248\2527123134.py:39: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  player_data_merged = player_data_merged.fillna(method='ffill')


In [55]:
# Remove players with fewer than 6 games
player_counts = player_data_merged['PLAYER_ID'].value_counts()
valid_players = player_counts[player_counts >= 12].index
player_data_merged = player_data_merged[player_data_merged['PLAYER_ID'].isin(valid_players)]

In [56]:
player_data_merged

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_ID,PLAYER_NAME,MIN,...,avg_stl,avg_blk,avg_to,avg_pf,avg_fg_pct,avg_fg3_pct,avg_ft_pct,avg_plus_minus,avg_ts_pct,avg_efg_pct
245097,245097,245097,245097,471641,11400039,1610612751,BKN,708,Kevin Garnett,10.683333,...,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000
243251,243251,243251,243251,469795,21400006,1610612751,BKN,708,Kevin Garnett,22.683333,...,0.000000,0.000000,1.000000,1.0,0.75000,0.000000,0.000000,0.000000,0.750000,0.75000
242736,242736,242736,242736,469280,21400033,1610612751,BKN,708,Kevin Garnett,35.033333,...,0.000000,0.000000,2.500000,1.5,0.68750,0.000000,0.000000,-5.500000,0.687500,0.68750
242464,242464,242464,242464,469008,21400044,1610612751,BKN,708,Kevin Garnett,19.733333,...,0.666667,0.333333,2.666667,2.0,0.61400,0.000000,0.333333,1.333333,0.637333,0.61400
242189,242189,242189,242189,468733,21400060,1610612751,BKN,708,Kevin Garnett,25.133333,...,0.750000,0.250000,2.250000,2.5,0.55425,0.000000,0.250000,4.000000,0.571750,0.55425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2423,2423,2423,2423,2423,22200372,1610612762,UTA,1631323,Simone Fontecchio,19.566667,...,0.200000,0.300000,0.600000,0.9,0.37830,0.264833,0.180000,-4.400000,0.501500,0.53000
809,809,809,809,809,22200442,1610612762,UTA,1631323,Simone Fontecchio,14.866667,...,0.100000,0.400000,0.600000,0.9,0.41330,0.277211,0.240000,-3.800000,0.550300,0.58000
394,394,394,394,394,22200452,1610612762,UTA,1631323,Simone Fontecchio,10.316667,...,0.100000,0.400000,0.700000,0.9,0.43830,0.263350,0.240000,-5.700000,0.575300,0.60500
284,284,284,284,284,22200461,1610612762,UTA,1631323,Simone Fontecchio,5.066667,...,0.200000,0.300000,0.600000,0.9,0.41330,0.266667,0.240000,-4.400000,0.537800,0.56750


In [57]:
# Drop all rows where the SEASON is 2004
NBA_player_data = player_data_merged[player_data_merged['SEASON'] != 2014]

In [58]:
NBA_player_data

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_ID,PLAYER_NAME,MIN,...,avg_stl,avg_blk,avg_to,avg_pf,avg_fg_pct,avg_fg3_pct,avg_ft_pct,avg_plus_minus,avg_ts_pct,avg_efg_pct
215282,215282,215282,215282,441826,11500018,1610612750,MIN,708,Kevin Garnett,7.866667,...,0.9,1.0,0.7,2.3,0.5431,0.020833,0.20,1.4,0.5445,0.5431
214883,214883,214883,214883,441427,11500037,1610612750,MIN,708,Kevin Garnett,10.516667,...,1.0,1.0,0.5,1.9,0.5002,0.020408,0.20,1.4,0.5042,0.5002
214387,214387,214387,214387,440931,11500058,1610612750,MIN,708,Kevin Garnett,12.083333,...,0.7,1.0,0.3,1.9,0.5002,0.020000,0.10,-0.9,0.4999,0.5002
213903,213903,213903,213903,440447,11500077,1610612750,MIN,708,Kevin Garnett,16.783333,...,0.7,0.7,0.2,1.8,0.4402,0.019608,0.10,-2.4,0.4399,0.4402
213624,213624,213624,213624,440168,11500088,1610612750,MIN,708,Kevin Garnett,13.950000,...,0.9,0.6,0.2,2.1,0.4602,0.019231,0.10,-3.2,0.4599,0.4602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2423,2423,2423,2423,2423,22200372,1610612762,UTA,1631323,Simone Fontecchio,19.566667,...,0.2,0.3,0.6,0.9,0.3783,0.264833,0.18,-4.4,0.5015,0.5300
809,809,809,809,809,22200442,1610612762,UTA,1631323,Simone Fontecchio,14.866667,...,0.1,0.4,0.6,0.9,0.4133,0.277211,0.24,-3.8,0.5503,0.5800
394,394,394,394,394,22200452,1610612762,UTA,1631323,Simone Fontecchio,10.316667,...,0.1,0.4,0.7,0.9,0.4383,0.263350,0.24,-5.7,0.5753,0.6050
284,284,284,284,284,22200461,1610612762,UTA,1631323,Simone Fontecchio,5.066667,...,0.2,0.3,0.6,0.9,0.4133,0.266667,0.24,-4.4,0.5378,0.5675


In [59]:
NBA_player_data.columns

Index(['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'GAME_ID',
       'TEAM_ID', 'TEAM_ABBREVIATION', 'PLAYER_ID', 'PLAYER_NAME', 'MIN',
       'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS',
       'PLUS_MINUS', 'SEASON', 'IS_HOME', 'TEAM_WINS', 'TEAM_CITY', 'TS%',
       'GAME_DATE_EST', 'eFG%', 'avg_min', 'avg_pts', 'avg_reb', 'avg_ast',
       'avg_stl', 'avg_blk', 'avg_to', 'avg_pf', 'avg_fg_pct', 'avg_fg3_pct',
       'avg_ft_pct', 'avg_plus_minus', 'avg_ts_pct', 'avg_efg_pct'],
      dtype='object')

In [60]:
NBA_player_data.drop([ 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0','MIN','FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 
       'PLUS_MINUS','TS%'] , axis=1 , inplace=True)

C:\Users\Daphne\AppData\Local\Temp\ipykernel_22248\2922370485.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NBA_player_data.drop([ 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0','MIN','FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',


In [28]:
NBA_player_data

,Unnamed: 0.3,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_ID,PLAYER_NAME,PTS,SEASON,IS_HOME,TEAM_WINS,...,avg_stl,avg_blk,avg_to,avg_pf,avg_fg_pct,avg_fg3_pct,avg_ft_pct,avg_plus_minus,avg_ts_pct,avg_efg_pct
215282,215282,11500018,1610612750,MIN,708,Kevin Garnett,0,2015,1,0,...,0.9,1.0,0.7,2.3,0.5431,0.020833,0.20,1.4,0.5445,0.5431
214883,214883,11500037,1610612750,MIN,708,Kevin Garnett,2,2015,1,0,...,1.0,1.0,0.5,1.9,0.5002,0.020408,0.20,1.4,0.5042,0.5002
214387,214387,11500058,1610612750,MIN,708,Kevin Garnett,0,2015,1,1,...,0.7,1.0,0.3,1.9,0.5002,0.020000,0.10,-0.9,0.4999,0.5002
213903,213903,11500077,1610612750,MIN,708,Kevin Garnett,8,2015,0,0,...,0.7,0.7,0.2,1.8,0.4402,0.019608,0.10,-2.4,0.4399,0.4402
213624,213624,11500088,1610612750,MIN,708,Kevin Garnett,2,2015,0,0,...,0.9,0.6,0.2,2.1,0.4602,0.019231,0.10,-3.2,0.4599,0.4602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2423,2423,22200372,1610612762,UTA,1631323,Simone Fontecchio,18,2022,1,1,...,0.2,0.3,0.6,0.9,0.3783,0.264833,0.18,-4.4,0.5015,0.5300
809,809,22200442,1610612762,UTA,1631323,Simone Fontecchio,2,2022,0,0,...,0.1,0.4,0.6,0.9,0.4133,0.277211,0.24,-3.8,0.5503,0.5800
394,394,22200452,1610612762,UTA,1631323,Simone Fontecchio,3,2022,0,0,...,0.1,0.4,0.7,0.9,0.4383,0.263350,0.24,-5.7,0.5753,0.6050
284,284,22200461,1610612762,UTA,1631323,Simone Fontecchio,0,2022,0,1,...,0.2,0.3,0.6,0.9,0.4133,0.266667,0.24,-4.4,0.5378,0.5675


In [62]:
dummy_two=pd.read_excel(r"C:\Users\Daphne\OneDrive\Documents\NBA data set\player_data_updates.xlsx")

In [63]:
dummy_two.columns

Index(['Unnamed: 0', 'GAME_ID', 'SEASON', 'GAME_DATE_EST', 'TEAM_ID',
       'avg_pts', 'avg_fg_pct', 'avg_ft_pct', 'avg_fg3_pct', 'avg_ast',
       'avg_reb', 'avg_stl', 'avg_to', 'avg_pf', 'avg_fga', 'avg_efg_pct',
       'avg_fgm', 'win_streak', 'home_avg_pts', 'away_avg_pts', 'home_win_pct',
       'away_win_pct', 'home_head_to_head_avg_pts',
       'home_head_to_head_win_pct', 'Unnamed: 23'],
      dtype='object')

In [64]:
dummy_two.drop_duplicates(subset=['GAME_ID','TEAM_ID'])

,Unnamed: 0,GAME_ID,SEASON,GAME_DATE_EST,TEAM_ID,avg_pts,avg_fg_pct,avg_ft_pct,avg_fg3_pct,avg_ast,...,avg_efg_pct,avg_fgm,win_streak,home_avg_pts,away_avg_pts,home_win_pct,away_win_pct,home_head_to_head_avg_pts,home_head_to_head_win_pct,Unnamed: 23
0,NaN,10500003,2005,2005-10-11,1610612737,97.5,0.462,0.676,0.367,20.6,...,0.493,37.4,0.0,111.0,96.2,0.8,0.2,100.340,0.474,NaN
1,1.0,10500015,2005,2005-10-12,1610612737,96.9,0.452,0.758,0.305,22.6,...,0.469,36.5,1.0,111.0,92.8,0.8,0.2,100.247,0.447,NaN
2,2.0,10500042,2005,2005-10-17,1610612737,95.2,0.444,0.745,0.272,22.3,...,0.459,36.3,1.0,95.6,92.8,0.4,0.2,100.247,0.447,NaN
3,3.0,10500047,2005,2005-10-18,1610612737,87.3,0.417,0.766,0.304,20.3,...,0.446,33.1,0.0,85.0,92.8,0.0,0.2,99.617,0.404,NaN
4,4.0,10500074,2005,2005-10-22,1610612737,94.1,0.456,0.654,0.326,19.0,...,0.484,35.5,0.0,98.8,92.8,0.4,0.2,100.340,0.474,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47745,47745.0,22200414,2022,2022-12-14,1610612766,110.8,0.461,0.758,0.378,23.7,...,0.536,40.6,0.0,114.2,108.8,0.8,0.4,101.268,0.493,NaN
47746,47746.0,22200428,2022,2022-12-16,1610612766,111.0,0.464,0.792,0.318,23.8,...,0.522,42.2,0.0,114.2,111.4,0.8,0.2,101.435,0.553,NaN
47747,47747.0,22200450,2022,2022-12-18,1610612766,117.8,0.533,0.736,0.359,28.5,...,0.596,45.1,0.0,124.0,111.4,0.8,0.2,105.750,0.611,NaN
47748,47748.0,22200460,2022,2022-12-19,1610612766,115.4,0.483,0.806,0.343,26.1,...,0.557,41.7,2.0,126.2,111.4,0.8,0.2,107.703,0.378,NaN


In [65]:


# Group the dataset by GAME_ID and collect all TEAM_IDs for each game
game_to_teams = dummy_two.groupby('GAME_ID')['TEAM_ID'].apply(list).to_dict()

# Define a function to get the opposition team ID
def get_opposition_team_id(row):
    teams = game_to_teams[row['GAME_ID']]
    # Return the team ID that is not the current TEAM_ID
    return teams[1] if teams[0] == row['TEAM_ID'] else teams[0]

# Apply the function to create a new column for the opposition team ID
dummy_two['OPP_TEAM_ID'] = dummy_two.apply(get_opposition_team_id, axis=1)




In [66]:
dummy_two

,Unnamed: 0,GAME_ID,SEASON,GAME_DATE_EST,TEAM_ID,avg_pts,avg_fg_pct,avg_ft_pct,avg_fg3_pct,avg_ast,...,avg_fgm,win_streak,home_avg_pts,away_avg_pts,home_win_pct,away_win_pct,home_head_to_head_avg_pts,home_head_to_head_win_pct,Unnamed: 23,OPP_TEAM_ID
0,NaN,10500003,2005,2005-10-11,1610612737,97.500,0.462,0.676,0.367,20.600,...,37.4,0.000,111.0,96.2,0.8,0.2,100.340,0.474,NaN,1610612753
1,1.0,10500015,2005,2005-10-12,1610612737,96.900,0.452,0.758,0.305,22.600,...,36.5,1.000,111.0,92.8,0.8,0.2,100.247,0.447,NaN,1610612766
2,2.0,10500042,2005,2005-10-17,1610612737,95.200,0.444,0.745,0.272,22.300,...,36.3,1.000,95.6,92.8,0.4,0.2,100.247,0.447,NaN,1610612766
3,3.0,10500047,2005,2005-10-18,1610612737,87.300,0.417,0.766,0.304,20.300,...,33.1,0.000,85.0,92.8,0.0,0.2,99.617,0.404,NaN,1610612740
4,4.0,10500074,2005,2005-10-22,1610612737,94.100,0.456,0.654,0.326,19.000,...,35.5,0.000,98.8,92.8,0.4,0.2,100.340,0.474,NaN,1610612753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95495,95495.0,22200414,2022,2022-12-14,1610612765,99.845,0.507,116.900,0.486,0.804,...,89.2,0.560,43.3,5.0,121.0,113.8,0.800,0.600,NaN,1610612766
95496,95496.0,22200428,2022,2022-12-16,1610612737,100.247,0.447,120.400,0.495,0.807,...,89.2,0.572,44.1,6.0,121.0,120.4,0.800,0.800,NaN,1610612766
95497,95497.0,22200450,2022,2022-12-18,1610612743,101.472,0.389,122.800,0.505,0.800,...,88.1,0.587,44.4,7.0,126.2,120.4,1.000,0.800,NaN,1610612766
95498,95498.0,22200460,2022,2022-12-19,1610612758,107.108,0.622,123.900,0.508,0.771,...,87.9,0.591,44.6,8.0,123.6,120.4,1.000,0.800,NaN,1610612766


In [67]:
#Team data that is taken to merge the defence metrics for the players data
dummy_two.columns

Index(['Unnamed: 0', 'GAME_ID', 'SEASON', 'GAME_DATE_EST', 'TEAM_ID',
       'avg_pts', 'avg_fg_pct', 'avg_ft_pct', 'avg_fg3_pct', 'avg_ast',
       'avg_reb', 'avg_stl', 'avg_to', 'avg_pf', 'avg_fga', 'avg_efg_pct',
       'avg_fgm', 'win_streak', 'home_avg_pts', 'away_avg_pts', 'home_win_pct',
       'away_win_pct', 'home_head_to_head_avg_pts',
       'home_head_to_head_win_pct', 'Unnamed: 23', 'OPP_TEAM_ID'],
      dtype='object')

In [68]:
# merge the defence metrics 
total_merged=NBA_player_data.merge(dummy_two,left_on=['GAME_ID','TEAM_ID'], right_on=['GAME_ID','OPP_TEAM_ID'],how = 'left')

In [69]:
total_merged.columns

Index(['Unnamed: 0.3', 'GAME_ID', 'TEAM_ID_x', 'TEAM_ABBREVIATION',
       'PLAYER_ID', 'PLAYER_NAME', 'PTS', 'SEASON_x', 'IS_HOME', 'TEAM_WINS',
       'TEAM_CITY', 'GAME_DATE_EST_x', 'eFG%', 'avg_min', 'avg_pts_x',
       'avg_reb_x', 'avg_ast_x', 'avg_stl_x', 'avg_blk', 'avg_to_x',
       'avg_pf_x', 'avg_fg_pct_x', 'avg_fg3_pct_x', 'avg_ft_pct_x',
       'avg_plus_minus', 'avg_ts_pct', 'avg_efg_pct_x', 'Unnamed: 0',
       'SEASON_y', 'GAME_DATE_EST_y', 'TEAM_ID_y', 'avg_pts_y', 'avg_fg_pct_y',
       'avg_ft_pct_y', 'avg_fg3_pct_y', 'avg_ast_y', 'avg_reb_y', 'avg_stl_y',
       'avg_to_y', 'avg_pf_y', 'avg_fga', 'avg_efg_pct_y', 'avg_fgm',
       'win_streak', 'home_avg_pts', 'away_avg_pts', 'home_win_pct',
       'away_win_pct', 'home_head_to_head_avg_pts',
       'home_head_to_head_win_pct', 'Unnamed: 23', 'OPP_TEAM_ID'],
      dtype='object')

In [70]:
total_merged.drop(['SEASON_y',
       'GAME_DATE_EST_y', 'TEAM_ID_y', 'avg_pts_y', 'avg_fg_pct_y',
       'avg_ft_pct_y', 'avg_fg3_pct_y', 'avg_ast_y', 'avg_reb_y',
        'avg_pf_y', 'avg_fga', 'avg_efg_pct_y', 'avg_fgm',
       'win_streak', 'home_avg_pts', 'away_avg_pts', 'home_win_pct',
       'away_win_pct', 'home_head_to_head_avg_pts',
       'home_head_to_head_win_pct', 'Unnamed: 23', 'OPP_TEAM_ID'], axis=1,inplace=True)

In [71]:
NBA_player_data.columns

Index(['Unnamed: 0.3', 'GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'PLAYER_ID',
       'PLAYER_NAME', 'PTS', 'SEASON', 'IS_HOME', 'TEAM_WINS', 'TEAM_CITY',
       'GAME_DATE_EST', 'eFG%', 'avg_min', 'avg_pts', 'avg_reb', 'avg_ast',
       'avg_stl', 'avg_blk', 'avg_to', 'avg_pf', 'avg_fg_pct', 'avg_fg3_pct',
       'avg_ft_pct', 'avg_plus_minus', 'avg_ts_pct', 'avg_efg_pct'],
      dtype='object')

In [72]:
# merged data with opposite team defence 
total_merged

,Unnamed: 0.3,GAME_ID,TEAM_ID_x,TEAM_ABBREVIATION,PLAYER_ID,PLAYER_NAME,PTS,SEASON_x,IS_HOME,TEAM_WINS,...,avg_pf_x,avg_fg_pct_x,avg_fg3_pct_x,avg_ft_pct_x,avg_plus_minus,avg_ts_pct,avg_efg_pct_x,Unnamed: 0,avg_stl_y,avg_to_y
0,215282,11500018,1610612750,MIN,708,Kevin Garnett,0,2015,1,0,...,2.3,0.5431,0.020833,0.20,1.4,0.5445,0.5431,37671.0,7.2,14.1
1,215282,11500018,1610612750,MIN,708,Kevin Garnett,0,2015,1,0,...,2.3,0.5431,0.020833,0.20,1.4,0.5445,0.5431,69726.0,24.5,45.2
2,214883,11500037,1610612750,MIN,708,Kevin Garnett,2,2015,1,0,...,1.9,0.5002,0.020408,0.20,1.4,0.5042,0.5002,7451.0,7.5,13.5
3,214883,11500037,1610612750,MIN,708,Kevin Garnett,2,2015,1,0,...,1.9,0.5002,0.020408,0.20,1.4,0.5042,0.5002,69727.0,25.0,46.2
4,214387,11500058,1610612750,MIN,708,Kevin Garnett,0,2015,1,1,...,1.9,0.5002,0.020000,0.10,-0.9,0.4999,0.5002,39223.0,8.3,13.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428511,394,22200452,1610612762,UTA,1631323,Simone Fontecchio,3,2022,0,0,...,0.9,0.4383,0.263350,0.24,-5.7,0.5753,0.6050,89273.0,24.4,44.4
428512,284,22200461,1610612762,UTA,1631323,Simone Fontecchio,0,2022,0,1,...,0.9,0.4133,0.266667,0.24,-4.4,0.5378,0.5675,46232.0,6.5,15.9
428513,284,22200461,1610612762,UTA,1631323,Simone Fontecchio,0,2022,0,1,...,0.9,0.4133,0.266667,0.24,-4.4,0.5378,0.5675,89274.0,24.7,43.9
428514,47,22200478,1610612762,UTA,1631323,Simone Fontecchio,0,2022,1,1,...,0.9,0.3133,0.254545,0.19,-5.3,0.4314,0.4175,44671.0,6.8,15.3


In [73]:
total_merged.to_excel(r"C:\Users\Daphne\OneDrive\Documents\NBA data set\modelling_player_copy.xlsx")

In [74]:
Player_data=pd.read_excel(r"C:\Users\Daphne\OneDrive\Documents\NBA data set\modelling_player_copy.xlsx")

In [76]:
Player_data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0.3', 'GAME_ID', 'TEAM_ID_x',
       'TEAM_ABBREVIATION', 'PLAYER_ID', 'PLAYER_NAME', 'PTS', 'SEASON_x',
       'IS_HOME', 'TEAM_WINS', 'TEAM_CITY', 'GAME_DATE_EST_x', 'eFG%',
       'avg_min', 'avg_pts_x', 'avg_reb_x', 'avg_ast_x', 'avg_stl_x',
       'avg_blk', 'avg_to_x', 'avg_pf_x', 'avg_fg_pct_x', 'avg_fg3_pct_x',
       'avg_ft_pct_x', 'avg_plus_minus', 'avg_ts_pct', 'avg_efg_pct_x',
       'Unnamed: 0', 'avg_stl_y', 'avg_to_y'],
      dtype='object')

In [82]:
Player_data

,Unnamed: 0.1,Unnamed: 0.3,GAME_ID,TEAM_ID_x,TEAM_ABBREVIATION,PLAYER_ID,PLAYER_NAME,PTS,SEASON_x,IS_HOME,...,avg_pf_x,avg_fg_pct_x,avg_fg3_pct_x,avg_ft_pct_x,avg_plus_minus,avg_ts_pct,avg_efg_pct_x,Unnamed: 0,avg_stl_y,avg_to_y
0,0,215282,11500018,1610612750,MIN,708,Kevin Garnett,0,2015,1,...,2.3,0.5431,0.020833,0.2000,1.4,0.5445,0.5431,37671,7.2,14.1
1,1,215282,11500018,1610612750,MIN,708,Kevin Garnett,0,2015,1,...,2.3,0.5431,0.020833,0.2000,1.4,0.5445,0.5431,69726,24.5,45.2
2,1922,215289,11500018,1610612750,MIN,1889,Andre Miller,4,2015,1,...,1.4,0.3797,0.046931,0.2417,-0.5,0.3971,0.3797,37671,7.2,14.1
3,1923,215289,11500018,1610612750,MIN,1889,Andre Miller,4,2015,1,...,1.4,0.3797,0.046931,0.2417,-0.5,0.3971,0.3797,69726,24.5,45.2
4,21800,215287,11500018,1610612750,MIN,2755,Kevin Martin,9,2015,1,...,1.9,0.3739,0.387833,0.9417,-12.6,0.4972,0.4168,37671,7.2,14.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428511,418379,15958,22101143,1610612760,OKC,1630526,Jeremiah Robinson-Earl,9,2021,1,...,2.1,0.3233,0.247204,0.3500,-2.2,0.4273,0.4019,86038,28.2,47.7
428512,425348,15951,22101143,1610612760,OKC,1630598,Aaron Wiggins,13,2021,1,...,1.8,0.4441,0.317529,0.5342,-10.8,0.5262,0.4962,1561,7.3,14.5
428513,425349,15951,22101143,1610612760,OKC,1630598,Aaron Wiggins,13,2021,1,...,1.8,0.4441,0.317529,0.5342,-10.8,0.5262,0.4962,86038,28.2,47.7
428514,426824,15957,22101143,1610612760,OKC,1630846,Olivier Sarr,17,2021,1,...,2.7,0.4880,0.208333,0.2084,0.9,0.5663,0.5505,1561,7.3,14.5
